# Example Analysis

This notebook provides example queries and visualizations.

## Setup
The cell below is used to
* import required libraries
* setting up the connection to the Neo4j database
* define the D3 based HTML template for custom visualizations

In [1]:
import pandas as pd
import plotly.express as px
import pygal as pg
from string import Template
from IPython.core.display import display, HTML
from IPython.display import HTML, Javascript, display

neo4j_url=%env NEO4J_URL

%reload_ext cypher
%config CypherMagic.uri=neo4j_url + "/db/data"

def configure_d3():
    """Tell require where to get d3 from in `require(['d3'])`"""
    display(Javascript("""
    require.config({
      paths: {
        lodash: "/notebooks/vis/lib/lodash.min",
        d3: "/notebooks/vis/lib/d3.v4.min"
      }
    })"""))

configure_d3()

base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="/notebooks/vis/lib/svg.jquery.js"></script>
  <script type="text/javascript" src="/notebooks/vis/lib/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""

<IPython.core.display.Javascript object>

## Artifacts

In [12]:
%cypher MATCH \
          (a:Artifact)-[:CONTAINS]->(n:Type) \
        RETURN \
           a.fqn as Artifact, count(n) as TypesPerArtifact \
        ORDER BY \
           TypesPerArtifact desc

15 rows affected.


Artifact,TypesPerArtifact
org.opencwa:protocols:jar:2.2.0-SNAPSHOT,426
org.opencwa:distribution:jar:2.2.0-SNAPSHOT,171
org.opencwa:distribution:test-jar:2.2.0-SNAPSHOT,108
org.opencwa:persistence:jar:2.2.0-SNAPSHOT,51
org.opencwa:submission:jar:2.2.0-SNAPSHOT,34
org.opencwa:submission:test-jar:2.2.0-SNAPSHOT,30
org.opencwa:download:test-jar:2.2.0-SNAPSHOT,23
org.opencwa:persistence:test-jar:2.2.0-SNAPSHOT,21
org.opencwa:upload:jar:2.2.0-SNAPSHOT,21
org.opencwa:upload:test-jar:2.2.0-SNAPSHOT,19


In [13]:
artifactSize = %cypher MATCH (artifact:Main:Artifact)-[:CONTAINS]->(type:Type) \
                      RETURN artifact.fqn as Artifact, count(DISTINCT type) AS Types

df = artifactSize.get_dataframe()
fig = px.pie(df, names='Artifact', values='Types', title='Artifact Size')
fig.show()

8 rows affected.


## Artifact Dependencies

In [15]:
dependencies = %cypher MATCH (a1:Main:Artifact)-[:CONTAINS]->(t1:Type), \
                                    (a2:Main:Artifact)-[:CONTAINS]->(t2:Type), \
                                    (t1)-[dep:DEPENDS_ON]->(t2) \
                             WHERE  a1 <> a2 \
                             RETURN a1.name AS Source, \
                                    a2.name AS Target, \
                                    COUNT(dep) AS X_Count \
                             ORDER BY Source, Target

dependenciesData = dependencies.get_dataframe().to_csv(index = False).replace("\r\n","\\n").replace("\n","\\n")
text = Template(open('../vis/chord/chord-diagram.html', 'r').read().replace("\n","")).substitute({
    'chord_data': dependenciesData,
    'container': 'module-chord-diagram'})

HTML(text)

15 rows affected.


## Dependency Drilldown between  "distribution" and "persistence"

In [5]:
%cypher MATCH \
          (a1:Main:Artifact)-[:CONTAINS]->(t1:Type), \
          (a2:Main:Artifact)-[:CONTAINS]->(t2:Type), \
          (t1)-[dep:DEPENDS_ON]->(t2) \
        WHERE \
          a1.name = "distribution" \
          and a2.name = "persistence" \
        RETURN \
           t1.name as Dependent, t2.name as Dependency, dep.weight as Weight \
        ORDER BY \
          Weight desc

38 rows affected.


Dependent,Dependency,Weight
DiagnosisKeyBundler,DiagnosisKey,34
EnfParameterAdapter,DiagnosisKey,18
TestDataGeneration,DiagnosisKey,14
TemporaryExposureKeyExportFile,DiagnosisKey,12
ApplicationConfigurationV2PublicationConfig,PreDistributionTrlValueMappingProvider,12
ProdDiagnosisKeyBundler,DiagnosisKey,11
TraceTimeIntervalWarningExportFile,TraceTimeIntervalWarning,10
ProdTraceTimeIntervalWarningsPackageBundler,TraceTimeIntervalWarning,10
EnfParameterAdapter,DiagnosisKeyBuilders$FinalBuilder,9
TestDataGeneration,DiagnosisKeyBuilders$FinalBuilder,7


## Impact of changing DiagnosisKey

In [6]:
%cypher MATCH \
          (diagnosisKey:Type), \
          (a:Main:Artifact)-[:CONTAINS]->(t:Type), \
          (t)-[dep:DEPENDS_ON]->(diagnosisKey) \
        WHERE \
          diagnosisKey.name = "DiagnosisKey" \
        RETURN \
           a.name as Artifact, t.name as Dependents, dep.weight as Weight \
        ORDER BY \
           Artifact, Weight desc


31 rows affected.


Artifact,Dependents,Weight
distribution,DiagnosisKeyBundler,34
distribution,EnfParameterAdapter,18
distribution,TestDataGeneration,14
distribution,TemporaryExposureKeyExportFile,12
distribution,ProdDiagnosisKeyBundler,11
distribution,DemoDiagnosisKeyBundler,2
distribution,DiagnosisKeysStructureProvider,1
distribution,DiagnosisKeysHourDirectory,1
distribution,DiagnosisKeysDateDirectory,1
download,ValidFederationKeyFilter,10
